In [2]:
import numpy as np
import pandas as pd
import os
import cv2
import math
import random
import matplotlib.pyplot as plt
import shutil
from sklearn.preprocessing import QuantileTransformer
from PIL import Image
import torch
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Read dataset
df=pd.read_csv('/nf-ton-iot1/NF-ToN-IoT.csv')

In [4]:
df

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,192.168.1.195,63318,52.139.250.253,443,6,91.00,181,165,2,1,24,327,0,Benign
1,192.168.1.79,57442,192.168.1.255,15600,17,0.00,63,0,1,0,0,0,0,Benign
2,192.168.1.79,57452,239.255.255.250,15600,17,0.00,63,0,1,0,0,0,0,Benign
3,192.168.1.193,138,192.168.1.255,138,17,10.16,472,0,2,0,0,0,0,Benign
4,192.168.1.79,51989,192.168.1.255,15600,17,0.00,63,0,1,0,0,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379269,192.168.1.31,58032,192.168.1.194,80,6,7.00,216,180,4,3,18,9433,1,ddos
1379270,192.168.1.31,58034,192.168.1.194,80,6,7.00,216,180,4,3,18,9221,1,ddos
1379271,192.168.1.31,58036,192.168.1.194,80,6,7.00,216,180,4,3,18,9656,1,ddos
1379272,192.168.1.31,58038,192.168.1.194,80,6,7.00,216,180,4,3,18,10046,1,ddos


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['IPV4_SRC_ADDR'] = le.fit_transform(df['IPV4_SRC_ADDR'])
df['IPV4_DST_ADDR'] = le.fit_transform(df['IPV4_DST_ADDR'])

In [6]:
df = df.drop(['Label'],axis=1)

In [7]:
df

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Attack
0,72,63318,1239,443,6,91.00,181,165,2,1,24,327,Benign
1,84,57442,430,15600,17,0.00,63,0,1,0,0,0,Benign
2,84,57452,1123,15600,17,0.00,63,0,1,0,0,0,Benign
3,70,138,430,138,17,10.16,472,0,2,0,0,0,Benign
4,84,51989,430,15600,17,0.00,63,0,1,0,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379269,75,58032,428,80,6,7.00,216,180,4,3,18,9433,ddos
1379270,75,58034,428,80,6,7.00,216,180,4,3,18,9221,ddos
1379271,75,58036,428,80,6,7.00,216,180,4,3,18,9656,ddos
1379272,75,58038,428,80,6,7.00,216,180,4,3,18,10046,ddos


In [8]:
# Transform all features into the scale of [0,1]
numeric_features = df.dtypes[df.dtypes != 'object'].index
scaler = QuantileTransformer()
df[numeric_features] = scaler.fit_transform(df[numeric_features])

In [9]:
# Multiply the feature values by 255 to transform them into the scale of [0,255]
df[numeric_features] = df[numeric_features].apply(
    lambda x: (x*255))

In [10]:
df.describe()

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
count,1.379274e+06,1.379274e+06,1.379274e+06,1.379274e+06,1.379274e+06,1.379274e+06,1.379274e+06,1.379274e+06,1.379274e+06,1.379274e+06,1.379274e+06,1.379274e+06
mean,1.274728e+02,1.279630e+02,1.282030e+02,1.270070e+02,1.270623e+02,1.292112e+02,1.274559e+02,1.270728e+02,1.228455e+02,1.265599e+02,1.257135e+02,1.183113e+02
std,7.275159e+01,7.345642e+01,7.217140e+01,6.549244e+01,3.695393e+01,6.441791e+01,7.363601e+01,7.570382e+01,7.822732e+01,7.491853e+01,7.181666e+01,8.524988e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.083333e+01,6.465078e+01,8.500000e+01,9.150901e+01,1.163964e+02,9.635886e+01,6.751502e+01,6.662162e+01,7.466216e+01,6.496246e+01,7.772523e+01,0.000000e+00
50%,1.212462e+02,1.270410e+02,1.327327e+02,9.150901e+01,1.163964e+02,9.635886e+01,1.342643e+02,1.352853e+02,1.213739e+02,1.341366e+02,1.269895e+02,1.276276e+02
75%,1.876126e+02,1.917874e+02,1.853153e+02,1.879955e+02,1.163964e+02,2.164565e+02,1.913138e+02,1.872297e+02,1.685961e+02,1.883784e+02,1.868468e+02,1.912376e+02
max,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02


In [11]:
df.dtypes

IPV4_SRC_ADDR                 float64
L4_SRC_PORT                   float64
IPV4_DST_ADDR                 float64
L4_DST_PORT                   float64
PROTOCOL                      float64
L7_PROTO                      float64
IN_BYTES                      float64
OUT_BYTES                     float64
IN_PKTS                       float64
OUT_PKTS                      float64
TCP_FLAGS                     float64
FLOW_DURATION_MILLISECONDS    float64
Attack                         object
dtype: object

In [12]:
# The labels of the dataset. "R" indicates normal patterns, and there are four types of attack (DoS, fuzzy. gear spoofing, and RPM spoofing zttacks)
df.Attack.value_counts()

Attack
injection     468539
ddos          326345
Benign        270279
password      156299
xss            99944
scanning       21467
dos            17717
backdoor       17247
mitm            1295
ransomware       142
Name: count, dtype: int64

In [13]:
df0=df[df['Attack']=='Benign'].drop(['Attack'],axis=1)
df1=df[df['Attack']=='injection'].drop(['Attack'],axis=1)
df2=df[df['Attack']=='ddos'].drop(['Attack'],axis=1)
df3=df[df['Attack']=='password'].drop(['Attack'],axis=1)
df4=df[df['Attack']=='xss'].drop(['Attack'],axis=1)
df5=df[df['Attack']=='scanning'].drop(['Attack'],axis=1)
df6=df[df['Attack']=='dos'].drop(['Attack'],axis=1)
df7=df[df['Attack']=='backdoor'].drop(['Attack'],axis=1)
df8=df[df['Attack']=='mitm'].drop(['Attack'],axis=1)
df9=df[df['Attack']=='ransomware'].drop(['Attack'],axis=1)

In [14]:
df0

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
0,41.351351,253.831966,245.555556,187.995495,116.396396,216.456456,100.060060,112.057057,74.662162,64.962462,115.120120,182.507508
1,253.213213,223.038801,203.310811,217.349850,243.258258,0.000000,32.289790,0.000000,0.000000,0.000000,0.000000,0.000000
2,253.213213,223.109641,244.151652,217.349850,243.258258,0.000000,32.289790,0.000000,0.000000,0.000000,0.000000,0.000000
3,34.459459,8.945899,203.310811,166.426426,243.258258,182.350951,145.367868,0.000000,74.662162,0.000000,0.000000,0.000000
4,253.213213,176.855504,203.310811,217.349850,243.258258,0.000000,32.289790,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1379239,16.719219,19.782282,210.457958,240.535748,116.396396,216.456456,122.139640,0.000000,74.662162,0.000000,120.990991,238.943644
1379240,16.719219,19.782282,210.457958,240.513285,116.396396,216.456456,122.139640,0.000000,74.662162,0.000000,120.990991,238.190033
1379241,16.719219,19.782282,210.457958,240.529330,116.396396,216.456456,122.139640,0.000000,74.662162,0.000000,120.990991,238.943644
1379242,16.719219,19.782282,210.457958,240.503658,116.396396,216.456456,122.139640,0.000000,74.662162,0.000000,120.990991,237.585275


In [15]:
df1

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
12,121.246246,248.927343,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.000000,76.959459
14,121.246246,77.507943,21.186186,9.189189,243.258258,11.231231,44.924925,96.103604,74.662162,108.738739,0.000000,78.108108
16,121.246246,101.070137,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.000000,75.555556
20,121.246246,128.331068,21.186186,9.189189,243.258258,11.231231,44.924925,96.103604,74.662162,108.738739,0.000000,78.108108
23,121.246246,239.851965,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.000000,73.513514
...,...,...,...,...,...,...,...,...,...,...,...,...
1378906,70.833333,204.739408,132.732733,91.509009,116.396396,96.358859,67.515015,66.621622,74.662162,64.962462,77.725225,0.000000
1378909,70.833333,204.771279,132.732733,91.509009,116.396396,96.358859,67.515015,66.621622,74.662162,64.962462,77.725225,0.000000
1378912,70.833333,204.803150,132.732733,91.509009,116.396396,96.358859,114.935237,119.528439,168.596096,134.136637,106.441441,253.261767
1379052,121.246246,118.438438,21.186186,9.189189,243.258258,11.231231,39.819820,93.040541,0.000000,64.962462,0.000000,0.000000


In [16]:
df2

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
13,121.246246,248.927343,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.000000,76.959459
15,121.246246,77.507943,21.186186,9.189189,243.258258,11.231231,44.924925,96.103604,74.662162,108.738739,0.000000,78.108108
17,121.246246,101.070137,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.000000,75.555556
21,121.246246,128.331068,21.186186,9.189189,243.258258,11.231231,44.924925,96.103604,74.662162,108.738739,0.000000,78.108108
24,121.246246,239.851965,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.000000,73.513514
...,...,...,...,...,...,...,...,...,...,...,...,...
1379269,121.246246,227.796706,165.405405,91.509009,116.396396,96.358859,108.611111,114.099099,121.373874,119.587087,77.725225,240.425511
1379270,121.246246,227.814986,165.405405,91.509009,116.396396,96.358859,108.611111,114.099099,121.373874,119.587087,77.725225,240.148165
1379271,121.246246,227.833266,165.405405,91.509009,116.396396,96.358859,108.611111,114.099099,121.373874,119.587087,77.725225,240.757281
1379272,121.246246,227.851545,165.405405,91.509009,116.396396,96.358859,108.611111,114.099099,121.373874,119.587087,77.725225,241.709774


In [17]:
df3

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
146,121.246246,140.463928,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.000000,75.555556
362,121.246246,137.149066,132.732733,91.509009,116.396396,96.358859,150.855856,187.229730,168.596096,188.378378,186.846847,140.135135
445,121.246246,137.166227,132.732733,91.509009,116.396396,96.358859,177.019520,201.524024,168.596096,188.378378,186.846847,213.311511
448,121.246246,137.200550,132.732733,91.509009,116.396396,96.358859,202.034535,201.524024,219.264264,188.378378,186.846847,214.489016
454,121.246246,137.269195,132.732733,91.509009,116.396396,96.358859,190.548048,201.524024,168.596096,188.378378,186.846847,214.680039
...,...,...,...,...,...,...,...,...,...,...,...,...
1378908,70.833333,204.739408,132.732733,91.509009,116.396396,96.358859,67.515015,66.621622,74.662162,64.962462,77.725225,0.000000
1378911,70.833333,204.771279,132.732733,91.509009,116.396396,96.358859,67.515015,66.621622,74.662162,64.962462,77.725225,0.000000
1378914,70.833333,204.803150,132.732733,91.509009,116.396396,96.358859,114.935237,119.528439,168.596096,134.136637,106.441441,253.261767
1378926,70.833333,81.981285,21.186186,9.189189,243.258258,11.231231,39.819820,93.040541,0.000000,64.962462,0.000000,0.000000


In [18]:
df4

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
193664,215.307808,117.648654,2.552553,91.509009,116.396396,96.358859,174.977477,202.800300,219.264264,188.378378,186.846847,213.648649
193742,215.307808,145.171258,21.186186,9.189189,243.258258,11.231231,92.019520,108.355856,74.662162,108.738739,0.000000,0.000000
193827,215.307808,117.760374,2.552553,91.509009,116.396396,96.358859,146.388889,202.800300,168.596096,188.378378,186.846847,212.415297
193853,215.307808,117.793885,2.552553,91.509009,116.396396,96.358859,108.611111,102.102102,121.373874,108.738739,106.441441,209.823415
193859,215.307808,117.777130,2.552553,91.509009,116.396396,96.358859,108.611111,102.102102,121.373874,108.738739,106.441441,215.192684
...,...,...,...,...,...,...,...,...,...,...,...,...
765654,187.612613,65.535325,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.000000,81.426426
765657,187.612613,171.053988,21.186186,9.189189,243.258258,11.231231,44.924925,96.103604,74.662162,108.738739,0.000000,99.294294
765659,187.612613,236.090086,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.000000,92.402402
765661,187.612613,126.826082,21.186186,9.189189,243.258258,11.231231,44.924925,96.103604,74.662162,108.738739,0.000000,70.705706


In [19]:
df5

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
18,121.246246,128.331068,21.186186,9.189189,243.258258,11.231231,44.924925,96.103604,74.662162,108.738739,0.000000,78.108108
25,121.246246,114.233893,21.186186,9.189189,243.258258,11.231231,44.924925,96.103604,74.662162,108.738739,0.000000,79.384384
35,121.246246,142.836587,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.000000,73.513514
911,121.246246,139.625187,132.732733,91.509009,116.396396,96.358859,184.677177,187.229730,168.596096,188.378378,186.846847,76.959459
1643,121.246246,143.636095,132.732733,91.509009,116.396396,96.358859,188.378378,187.229730,168.596096,188.378378,186.846847,84.489489
...,...,...,...,...,...,...,...,...,...,...,...,...
1379176,70.833333,166.290675,21.186186,9.189189,243.258258,11.231231,35.608108,0.000000,0.000000,0.000000,0.000000,0.000000
1379195,70.833333,105.355056,21.186186,9.189189,243.258258,11.231231,35.608108,89.594595,0.000000,64.962462,0.000000,0.000000
1379204,121.246246,37.506710,21.186186,9.189189,243.258258,11.231231,35.608108,89.594595,0.000000,64.962462,0.000000,70.705706
1379207,121.246246,109.763780,21.186186,9.189189,243.258258,11.231231,39.819820,93.040541,0.000000,64.962462,0.000000,73.513514


In [20]:
df6

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
11,121.246246,248.927343,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.0,76.959459
19,121.246246,128.331068,21.186186,9.189189,243.258258,11.231231,44.924925,96.103604,74.662162,108.738739,0.0,78.108108
22,121.246246,239.851965,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.0,73.513514
26,121.246246,114.233893,21.186186,9.189189,243.258258,11.231231,44.924925,96.103604,74.662162,108.738739,0.0,79.384384
36,121.246246,142.836587,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.0,73.513514
...,...,...,...,...,...,...,...,...,...,...,...,...
1379246,70.833333,190.210623,21.186186,9.189189,243.258258,11.231231,39.819820,93.040541,0.000000,64.962462,0.0,73.513514
1379250,70.833333,126.970822,21.186186,9.189189,243.258258,11.231231,35.608108,89.594595,0.000000,64.962462,0.0,0.000000
1379253,70.833333,245.144391,21.186186,9.189189,243.258258,11.231231,39.819820,93.040541,0.000000,64.962462,0.0,0.000000
1379258,70.833333,60.090442,21.186186,9.189189,243.258258,11.231231,35.608108,89.594595,0.000000,64.962462,0.0,0.000000


In [21]:
df7

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
779784,238.408408,145.542417,0.461853,166.171171,243.258258,182.124625,42.500000,94.954955,0.000000,64.962462,0.000000,95.082583
779802,238.408408,31.141141,160.938438,237.216140,116.396396,0.000000,254.775287,203.732101,254.745721,251.518377,115.120120,247.506916
779804,34.459459,153.656984,235.728228,213.648649,116.396396,250.150150,250.747372,146.133634,253.213213,253.213213,111.291291,241.805800
779811,34.459459,153.665536,235.728228,213.648649,116.396396,250.150150,250.747372,146.133634,253.213213,253.213213,111.291291,242.237237
779831,34.459459,153.677759,235.728228,213.648649,116.396396,250.150150,250.747372,146.133634,253.213213,253.213213,111.291291,242.650691
...,...,...,...,...,...,...,...,...,...,...,...,...
878567,34.459459,174.871375,233.048048,213.648649,116.396396,250.150150,249.256757,146.133634,253.213213,253.213213,111.291291,241.676905
878569,34.459459,174.880031,233.048048,91.509009,116.396396,96.358859,249.256757,146.133634,253.213213,253.213213,111.291291,244.181964
878571,34.459459,174.888688,233.048048,213.648649,116.396396,250.150150,249.256757,146.133634,253.213213,253.213213,111.291291,245.276805
878577,34.459459,174.897345,233.048048,213.648649,116.396396,250.150150,238.097381,143.453453,251.426426,251.127098,111.291291,239.220776


In [22]:
df8

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
1389,121.246246,217.744542,21.186186,9.189189,243.258258,11.231231,92.019520,108.355856,74.662162,108.738739,0.0,73.513514
2749,121.246246,196.558461,21.186186,9.189189,243.258258,11.231231,46.456456,98.145646,74.662162,108.738739,0.0,75.555556
3447,121.246246,213.531530,21.186186,9.189189,243.258258,11.231231,93.933934,126.120114,74.662162,108.738739,0.0,78.108108
24241,121.246246,88.804445,21.186186,9.189189,243.258258,11.231231,93.551051,112.364205,74.662162,108.738739,0.0,0.000000
44073,121.246246,221.873809,21.186186,9.189189,243.258258,11.231231,92.019520,108.355856,74.662162,108.738739,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1304693,121.246246,206.421556,21.186186,9.189189,243.258258,11.231231,39.819820,93.040541,0.000000,64.962462,0.0,73.513514
1307057,121.246246,131.196444,21.186186,9.189189,243.258258,11.231231,35.608108,89.594595,0.000000,64.962462,0.0,0.000000
1348237,121.246246,84.271029,21.186186,9.189189,243.258258,11.231231,39.819820,93.040541,0.000000,64.962462,0.0,78.108108
1352068,121.246246,39.991076,21.186186,9.189189,243.258258,11.231231,35.608108,89.594595,0.000000,64.962462,0.0,0.000000


In [23]:
df9

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS
779991,164.001502,59.896708,160.938438,209.564779,116.396396,184.294294,253.082048,121.481269,238.536036,155.578078,255.000000,0.0
780003,164.001502,46.354103,160.938438,209.564779,116.396396,182.350951,169.234234,127.301121,233.558559,188.378378,255.000000,0.0
780008,164.001502,63.468819,160.938438,209.564779,116.396396,184.294294,253.082048,121.481269,238.536036,155.578078,255.000000,0.0
780010,164.001502,103.789447,160.938438,209.564779,116.396396,184.294294,253.082048,121.481269,238.536036,155.578078,255.000000,0.0
781054,238.408408,61.083784,160.938438,209.564779,116.396396,182.350951,169.234234,127.599271,233.558559,188.378378,255.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
834496,164.001502,70.420257,160.938438,209.564779,116.396396,184.294294,253.082048,121.481269,238.536036,155.578078,255.000000,0.0
834497,164.001502,89.671598,160.938438,209.564779,116.396396,184.294294,253.082048,121.481269,238.536036,155.578078,255.000000,0.0
834509,164.001502,38.492989,160.938438,209.564779,116.396396,182.350951,254.753446,148.406387,254.744909,251.304348,255.000000,0.0
834623,164.001502,93.258757,160.938438,209.564779,116.396396,184.294294,121.437437,106.186186,168.596096,108.738739,186.846847,0.0


In [24]:
# Generate 9*9 color images for class 0 (Normal)
count=0
ims = []

image_path = "train/0/"
os.makedirs(image_path)

for i in range(0, len(df0)):
    count=count+1
    if count<=9:
        im=df0.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(6,6,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []

In [25]:
# Generate 9*9 color images for class 1 (RPM spoofing)
count=0
ims = []

image_path = "train/1/"
os.makedirs(image_path)

for i in range(0, len(df1)):
    count=count+1
    if count<=9:
        im=df1.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(6,6,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []

In [26]:
# Generate 9*9 color images for class 2 (Gear spoofing)
count=0
ims = []

image_path = "train/2/"
os.makedirs(image_path)

for i in range(0, len(df2)):
    count=count+1
    if count<=9:
        im=df2.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(6,6,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []

In [27]:
# Generate 9*9 color images for class 3 (DoS attack)
count=0
ims = []

image_path = "train/3/"
os.makedirs(image_path)

for i in range(0, len(df3)):
    count=count+1
    if count<=9:
        im=df3.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(6,6,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []

In [28]:
# Generate 9*9 color images for class 4 (Fuzzy attack)
count=0
ims = []

image_path = "train/4/"
os.makedirs(image_path)

for i in range(0, len(df4)):
    count=count+1
    if count<=9:
        im=df4.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(6,6,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []

In [29]:
# Generate 9*9 color images for class 4 (Fuzzy attack)
count=0
ims = []

image_path = "train/5/"
os.makedirs(image_path)

for i in range(0, len(df5)):
    count=count+1
    if count<=9:
        im=df5.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(6,6,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []

In [30]:
# Generate 9*9 color images for class 4 (Fuzzy attack)
count=0
ims = []

image_path = "train/6/"
os.makedirs(image_path)

for i in range(0, len(df6)):
    count=count+1
    if count<=9:
        im=df6.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(6,6,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []

In [31]:
# Generate 9*9 color images for class 4 (Fuzzy attack)
count=0
ims = []

image_path = "train/7/"
os.makedirs(image_path)

for i in range(0, len(df7)):
    count=count+1
    if count<=9:
        im=df7.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(6,6,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []

In [32]:
# Generate 9*9 color images for class 4 (Fuzzy attack)
count=0
ims = []

image_path = "train/8/"
os.makedirs(image_path)

for i in range(0, len(df8)):
    count=count+1
    if count<=9:
        im=df8.iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(6,6,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []

In [33]:
# Generate 9*9 color images for class 4 (Fuzzy attack)
count=0
ims = []

image_path = "train/9/"
os.makedirs(image_path)

for i in range(0, len(df9)):
    count=count+1
    if count<=9:
        im=df9 .iloc[i].values
        ims=np.append(ims,im)
    else:
        ims=np.array(ims).reshape(6,6,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []

In [34]:
# Create folders to store images
Train_Dir='./train/'
Val_Dir='./test/'
allimgs=[]
for subdir in os.listdir(Train_Dir):
    for filename in os.listdir(os.path.join(Train_Dir,subdir)):
        filepath=os.path.join(Train_Dir,subdir,filename)
        allimgs.append(filepath)
print(len(allimgs)) # Print the total number of images

137921


In [35]:
#split a test set from the dataset, train/test size = 80%/20%
Numbers=len(allimgs)//5 	#size of test set (20%)

def mymovefile(srcfile,dstfile):
    if not os.path.isfile(srcfile):
        print ("%s not exist!"%(srcfile))
    else:
        fpath,fname=os.path.split(dstfile)    
        if not os.path.exists(fpath):
            os.makedirs(fpath)               
        shutil.move(srcfile,dstfile)          
        #print ("move %s -> %s"%(srcfile,dstfile))

In [36]:
Numbers

27584

In [37]:
# Create the test set
val_imgs=random.sample(allimgs,Numbers)
for img in val_imgs:
    dest_path=img.replace(Train_Dir,Val_Dir)
    mymovefile(img,dest_path)
print('Finish creating test set')

Finish creating test set


In [38]:
#resize the images 224*224 for better CNN training
def get_224(folder,dstdir):
    imgfilepaths=[]
    for root,dirs,imgs in os.walk(folder):
        for thisimg in imgs:
            thisimg_path=os.path.join(root,thisimg)
            imgfilepaths.append(thisimg_path)
    for thisimg_path in imgfilepaths:
        dir_name,filename=os.path.split(thisimg_path)
        dir_name=dir_name.replace(folder,dstdir)
        new_file_path=os.path.join(dir_name,filename)
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
        img=cv2.imread(thisimg_path)
        img=cv2.resize(img,(224,224))
        cv2.imwrite(new_file_path,img)
    print('Finish resizing'.format(folder=folder))

In [39]:
DATA_DIR_224='./train_224/'
get_224(folder='./train/',dstdir=DATA_DIR_224)

Finish resizing


In [40]:
DATA_DIR2_224='./test_224/'
get_224(folder='./test/',dstdir=DATA_DIR2_224)

Finish resizing
